Hap, sur, dis, neu, fea, sad, ang
각 줄은 각 감정에 대한 7개의 확률이 표시되어 있음. csv

input = multiple csvs output = result.csv
output = (1, hap) (2, sad)

In [45]:
import numpy as np

emotions_dae = ['hap', 'sur', 'dis', 'neu', 'fea', 'sad', 'ang']
# emotions_asim = ['hap', 'sur', 'dis', 'neu', 'fea', 'sad', 'ang']
emotions_asim = ['ang', 'dis', 'fea', 'hap', 'neu', 'sad', 'sur']

In [49]:
def ensemble_thres(row1, row2):
    '''
    method: threshold
    return value: labels among hap ~ ang 
    '''
    
    thres = 0.7
    acc1 = 0.40
    acc2 = 0.43
    
    index1 = np.argmax(row1)
    index2 = np.argmax(row2)
    prob1 = row1[index1]
    prob2 = row2[index2]
    
    if (prob1 > thres and prob2 > thres):
        if (prob1 > prob2):
            return emotions_dae[index1]
        return emotions_asim[index2]
    if (prob1 > thres):
        return emotions_dae[index1]
    if (prob2 > thres):
        return emotions_asim[index2]
    
    if (acc1 > acc2):
        return emotions_dae[index1]
    return emotions_asim[index2]

In [50]:
def ensemble_soft_vote(row1, row2):
    '''
    method: soft vote
    return value: labels among hap ~ ang

    we can also consider the probabilities of each classes
    returned by all the models,
    average these probabilities 
    and keep the class with the highest average probability 
    '''

    sums = {'hap': 0, 'sur': 0, 'dis': 0, 'neu': 0, 
            'fea': 0, 'sad': 0, 'ang': 0}
    # fill in sums dictionary
    for i in range(len(row1)):
        sums[emotions_dae[i]] += row1[i]
    for i in range(len(row2)):
        sums[emotions_asim[i]] += row2[i]
        
    emos = [(k, v) for k, v in sums.items()]
    
    max_sum = 0
    result_emo = ''
    for elem in emos:
        if (elem[1] > max_sum):
            max_sum = elem[1]
            result_emo = elem[0]
    return result_emo

In [ ]:
def ensemble_thres_combined_with_soft_vote(row1, row2):
    '''
    method: soft vote
    return value: labels among hap ~ ang
    '''
    
    thres = 0.7
    
    index1 = np.argmax(row1)
    index2 = np.argmax(row2)
    prob1 = row1[index1]
    prob2 = row2[index2]
    
    if (prob1 > thres and prob2 > thres):
        if (prob1 > prob2):
            return emotions_dae[index1]
        return emotions_asim[index2]
    if (prob1 > thres):
        return emotions_dae[index1]
    if (prob2 > thres):
        return emotions_asim[index2]
    
    return ensemble_soft_vote(row1, row2)

In [51]:
import csv

'''
model1 = dae
model2 = asim
'''
######################
dae_filename = 'test2Result_dae.csv'
asim_filename = 'test2Result_asim.csv'
index_filename = 'test2_index.csv'
submit_filename = 'submit_test2_thres.csv'
######################


model1 = open(dae_filename, 'r')  
csv1 = csv.reader(model1, delimiter=',')
model2 = open(asim_filename, 'r')
csv2 = csv.reader(model2, delimiter=',')

conversion = open(index_filename)
conv_csv = csv.reader(conversion, delimiter=',')

csv_output = open(submit_filename, 'w')

line_num = 0
for row1 in csv1: # type(row) == list
    for row2 in csv2:
        for num in conv_csv:
            row1 = list(map(lambda x: float(x), row1))
            row2 = list(map(lambda x: float(x), row2))
            emo = ensemble_thres(row1, row2)
            csv_output.write(num[0] + "," + emo + "\n")
            break
        break
    line_num += 1
        
        
model1.close()
model2.close()
csv_output.close()